In [48]:
import pandas as pd 

df = pd.read_csv('cleaned_url_data.csv')
df.head()

,URL,HTML,Title,Abstract,Publish_date,Authors,KMeansTags
0,https://deepmind.google/research/publications/...,Automated Discovery of Interpretable Cognitive...,Automated Discovery of Interpretable Cognitive...,A principal goal of computational neuroscience...,6 February 2025,"Pablo Castro Rivadeneira, Kim Stachenfeld, Kev...","['data', 'neural', 'networks', 'features', 'fe..."
1,https://deepmind.google/research/publications/...,Scaling Pre-training to One Hundred Billion Da...,Scaling Pre-training to One Hundred Billion Da...,We provide an empirical investigation of the p...,11 February 2025,"Xiao Wang, Ibrahim Alabdulmohsin, Daniel Salz,...","['vision', 'scale', 'image', 'dense', 'text']"
2,https://deepmind.google/research/publications/...,Delta Variances - Google DeepMind ...,Generate videos in Gemini and Whisk with Veo 2...,Decision makers may suffer from uncertainty in...,20 February 2025,"Simon Schmitt, John Shawe-Taylor, Hado van Has...","['planning', 'inference', 'energy', 'variation..."
3,https://deepmind.google/research/publications/...,Effective Kernel Fuzzing with Learned White-bo...,Veo 2 15 April 2025 Effectiv...,Kernel fuzzers rely heavily on program mutatio...,1 April 2025,"Sishuai Gong, Wang Rui, Deniz Altinbüken, Pedr...","['hash', 'kernel', 'keys', 'snowplow', 'covera..."
4,https://deepmind.google/research/publications/...,TIPS: Text-Image Pretraining with Spatial awar...,Whisk with Veo 2 15 April 2025 ...,While image-text representation learning has b...,10 March 2025,"Kevis-Kokitsi Maninis, Kaifeng Chen, Soham Gho...","['vision', 'scale', 'image', 'dense', 'text']"


have a way to standardize the text:

In [49]:
import re
def clean_text(text):

    text = text.lower()

    text = re.sub(r"[^\w\s-]", "", text)
    return text

Include tags and title in search

In [50]:
blobs = zip(df["Abstract"],df["Title"], df["KMeansTags"])
print(type(df["KMeansTags"][0]))
blobs_to_search = [f"Title: {title} Abstract: {abstract}  Tags: {tags}" for abstract, title, tags in blobs]

blobs_to_search = [clean_text(blob) for blob in blobs_to_search]
print(blobs_to_search[0])


<class 'str'>
title automated discovery of interpretable cognitive programs underlying reward-guided behavior abstract a principal goal of computational neuroscience is to discover mathematical models that describe how the brain implements cognitive processes computational neuroscientists typically face a difficult choice between symbolic model-driven approaches that are interpretable but underperform at predicting animal behavior versus data-driven approaches that train overparameterized models to make accurate behavioral predictions with often hard-to-interpret blackbox models in this work we explore the use of an automated llm-based program discovery tool funsearch citeromeraparedes2024mathematical to discover novel symbolic programs that accurately describe animal behavior without sacrificing interpretability we dub this cogfunsearch and verify its efficacy on behavioral data from rodent reward-guided choice behaviors citemiller2021predictive we find that cogfunsearch is able to re

Creating a semantic search following this: https://spotintelligence.com/2023/10/17/semantic-search/


In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(blobs_to_search)

Technically i think it works without the tag boost thing, but i want to use tags sooo

In [58]:
from sklearn.metrics.pairwise import cosine_similarity

def search(querry, num_results):
    tag_boost = 0.1
    querry = clean_text(querry)
    query_vector = tfidf_vectorizer.transform([querry])
    # Calculate cosine similarity between the user query and all documents
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix)
    cosine_similarities = cosine_similarities.flatten()


    print(cosine_similarities.shape)
    results = []

    # Add a bonus if tags match search
    for i, tags in enumerate(df["KMeansTags"]):
        tags = clean_text(tags).split(' ')
        if set(querry.split(' ')) & set(tags):
            cosine_similarities[i] += tag_boost  

    # Get the index of the most similar document
    most_similar_document_index = cosine_similarities.argsort()
    # reverse array
    most_similar_document_index = most_similar_document_index[::-1]

    for idx in range(num_results):
        result = {
            "url": df["URL"][idx],
            "title": df["Title"][idx],
            "authors": df["Authors"][idx],
            "tags": df["KMeansTags"][idx]
        }
        results.append(result)
    

    return results


In [ ]:
print(search("automated discovery of interpretable cognitive programs underlying reward-guided behavior ", 5))|

(278,)
[{'url': 'https://deepmind.google/research/publications/130468', 'title': 'Automated Discovery of Interpretable Cognitive Programs underlying Reward-guided behavior', 'authors': 'Pablo Castro Rivadeneira, Kim Stachenfeld, Kevin Miller, Nenad Tomašev, Navodita Sharma, Ankit Anand, Alexander Novikov, Kuba Perlin, Nathaniel Daw, Will Dabney, noemielteto , Siddhant Jain, Kyle Levin', 'tags': "['data', 'neural', 'networks', 'features', 'feature']"}, {'url': 'https://deepmind.google/research/publications/132991', 'title': 'Scaling Pre-training to One Hundred Billion Data for Vision Language Models', 'authors': 'Xiao Wang, Ibrahim Alabdulmohsin, Daniel Salz, Zhe Li, Keran Rong, Xiaohua Zhai*  *', 'tags': "['vision', 'scale', 'image', 'dense', 'text']"}, {'url': 'https://deepmind.google/research/publications/112791', 'title': 'Generate videos in Gemini and Whisk with Veo 2 15 April 2025                   Delta Variances', 'authors': 'Simon Schmitt, John Shawe-Taylor, Hado van Hasselt', 